<h1><center>Inteligencia Artificial</center></h1>


<h2><center>Tarea 2 - Aspiradora Inteligente</center></h2>

---

#### El **objetivo** de esta tarea será implementar algoritmos de búsqueda para que la aspiradora pueda "ser inteligente".

* Con base a lo que se ha visto del perro robot y lo respectivo de los agentes, ¿será posible realizar el mismo ejercicio para que una aspiradora en un espacio de 2D (6x6) y pueda concluir de limpiar? Los resultados se deben imprimir en 'consola'.

* Además, deberán agregar su informacción correspondiente del equipo que hayan formado, así como el link de github correspondiente al cual cargarán la notebook (ejecutada previamente), para visualizar las respuestas.

---
<h2><center>Equipo 3</center></h2>
<h3>Axel Seyer</h3> 
<h3>Luis Alfredo Reynoso Gomez</h3>

---

### Modifiquen las clases que se han definido hasta este momento del Parque en 2D y el comportamiento del perro:

* cuarto = Cuarto2D(6,6, color={'Aqui la aspiradora': (50,50,50), 'Basura': (200, 200, 200)}) (por ejemplo)

* Actualización 1: añadir obstáculos y que la aspiradora determine aleatoriamente a donde moverse para continuar.

* Actualización 2: consistirá en la implementación de un algoritmo de búsqueda para determinar cuál sería un camino para limpiar la habitación (todavía sin obstáculos), expandiendo a una habitación de 10x10.

* Actualización 3: añadir obstáculos a dicha habitación para que realice su proceso de limpieza.


In [ ]:
#!pip install qpsolvers


In [1]:
from agents import *
from notebook import psource

In [2]:
class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # Do we need this?
        pass


class Agent(Thing):
    """An Agent is a subclass of Thing with one required instance attribute 
    (aka slot), .program, which should hold a function that takes one argument,
    the percept, and returns an action. (What counts as a percept or action 
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method, then the
    program could 'cheat' and look at aspects of the agent. It's not supposed
    to do that: the program can only look at the percepts. An agent program
    that needs a model of the world (and of the agent itself) will have to
    build and maintain its own model. There is an optional slot, .performance,
    which is a number giving the performance measure of the agent in its
    environment."""

    def __init__(self, program=None):
        self.alive = True
        self.bump = False
        self.holding = []
        self.performance = 0
        if program is None or not isinstance(program, collections.abc.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

    def can_grab(self, thing):
        """Return True if this agent can grab this thing.
        Override for appropriate subclasses of Agent and Thing."""
        return False

In [3]:
class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        if isinstance(location, numbers.Number):
            return [thing for thing in self.things
                    if thing.location == location and isinstance(thing, tclass)]
        return [thing for thing in self.things
                if all(x == y for x, y in zip(thing.location, location)) and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

In [4]:
class Direction:
    """A direction class for agents that want to move in a 2D plane
        Usage:
            d = Direction("down")
            To change directions:
            d = d + "right" or d = d + Direction.R #Both do the same thing
            Note that the argument to __add__ must be a string and not a Direction object.
            Also, it (the argument) can only be right or left."""

    R = "right"
    L = "left"
    U = "up"
    D = "down"

    def __init__(self, direction):
        self.direction = direction

    def __add__(self, heading):
        """
        >>> d = Direction('right')
        >>> l1 = d.__add__(Direction.L)
        >>> l2 = d.__add__(Direction.R)
        >>> l1.direction
        'up'
        >>> l2.direction
        'down'
        >>> d = Direction('down')
        >>> l1 = d.__add__('right')
        >>> l2 = d.__add__('left')
        >>> l1.direction == Direction.L
        True
        >>> l2.direction == Direction.R
        True
        """
        if self.direction == self.R:
            return {
                self.R: Direction(self.D),
                self.L: Direction(self.U),
            }.get(heading, None)
        elif self.direction == self.L:
            return {
                self.R: Direction(self.U),
                self.L: Direction(self.D),
            }.get(heading, None)
        elif self.direction == self.U:
            return {
                self.R: Direction(self.R),
                self.L: Direction(self.L),
            }.get(heading, None)
        elif self.direction == self.D:
            return {
                self.R: Direction(self.L),
                self.L: Direction(self.R),
            }.get(heading, None)

    def move_forward(self, from_location):
        """
        >>> d = Direction('up')
        >>> l1 = d.move_forward((0, 0))
        >>> l1
        (0, -1)
        >>> d = Direction(Direction.R)
        >>> l1 = d.move_forward((0, 0))
        >>> l1
        (1, 0)
        """
        # get the iterable class to return
        iclass = from_location.__class__
        x, y = from_location
        if self.direction == self.R:
            return iclass((x + 1, y))
        elif self.direction == self.L:
            return iclass((x - 1, y))
        elif self.direction == self.U:
            return iclass((x, y - 1))
        elif self.direction == self.D:
            return iclass((x, y + 1))


class XYEnvironment(Environment):
    """This class is for environments on a 2D plane, with locations
    labelled by (x, y) points, either discrete or continuous.

    Agents perceive things within a radius. Each agent in the
    environment has a .location slot which should be a location such
    as (0, 1), and a .holding slot, which should be a list of things
    that are held."""

    def __init__(self, width=10, height=10):
        super().__init__()

        self.width = width
        self.height = height
        self.observers = []
        # Sets iteration start and end (no walls).
        self.x_start, self.y_start = (0, 0)
        self.x_end, self.y_end = (self.width, self.height)

    perceptible_distance = 1

    def things_near(self, location, radius=None):
        """Return all things within radius of location."""
        if radius is None:
            radius = self.perceptible_distance
        radius2 = radius * radius
        return [(thing, radius2 - distance_squared(location, thing.location))
                for thing in self.things if distance_squared(
                location, thing.location) <= radius2]

    def percept(self, agent):
        """By default, agent perceives things within a default radius."""
        return self.things_near(agent.location)

    def execute_action(self, agent, action):
        agent.bump = False
        if action == 'TurnRight':
            agent.direction += Direction.R
        elif action == 'TurnLeft':
            agent.direction += Direction.L
        elif action == 'Forward':
            agent.bump = self.move_to(agent, agent.direction.move_forward(agent.location))
        elif action == 'Grab':
            things = [thing for thing in self.list_things_at(agent.location) if agent.can_grab(thing)]
            if things:    
                agent.holding.append(things[0])
                print("Grabbing ", things[0].__class__.__name__)
                self.delete_thing(things[0])
        elif action == 'Release':
            if agent.holding:
                dropped = agent.holding.pop()
                print("Dropping ", dropped.__class__.__name__)
                self.add_thing(dropped, location=agent.location)

    def default_location(self, thing):
        location = self.random_location_inbounds()
        while self.some_things_at(location, Obstacle):
            # we will find a random location with no obstacles
            location = self.random_location_inbounds()
        return location

    def move_to(self, thing, destination):
        """Move a thing to a new location. Returns True on success or False if there is an Obstacle.
        If thing is holding anything, they move with him."""
        thing.bump = self.some_things_at(destination, Obstacle)
        if not thing.bump:
            thing.location = destination
            for o in self.observers:
                o.thing_moved(thing)
            for t in thing.holding:
                self.delete_thing(t)
                self.add_thing(t, destination)
                t.location = destination
        return thing.bump

    def add_thing(self, thing, location=None, exclude_duplicate_class_items=False):
        """Add things to the world. If (exclude_duplicate_class_items) then the item won't be
        added if the location has at least one item of the same class."""
        if location is None:
            super().add_thing(thing)
        elif self.is_inbounds(location):
            if (exclude_duplicate_class_items and
                    any(isinstance(t, thing.__class__) for t in self.list_things_at(location))):
                return
            super().add_thing(thing, location)

    def is_inbounds(self, location):
        """Checks to make sure that the location is inbounds (within walls if we have walls)"""
        x, y = location
        return not (x < self.x_start or x > self.x_end or y < self.y_start or y > self.y_end)

    def random_location_inbounds(self, exclude=None):
        """Returns a random location that is inbounds (within walls if we have walls)"""
        location = (random.randint(self.x_start, self.x_end),
                    random.randint(self.y_start, self.y_end))
        if exclude is not None:
            while location == exclude:
                location = (random.randint(self.x_start, self.x_end),
                            random.randint(self.y_start, self.y_end))
        return location

    def delete_thing(self, thing):
        """Deletes thing, and everything it is holding (if thing is an agent)"""
        if isinstance(thing, Agent):
            del thing.holding

        super().delete_thing(thing)
        for obs in self.observers:
            obs.thing_deleted(thing)

    def add_walls(self):
        """Put walls around the entire perimeter of the grid."""
        for x in range(self.width):
            self.add_thing(Wall(), (x, 0))
            self.add_thing(Wall(), (x, self.height - 1))
        for y in range(1, self.height - 1):
            self.add_thing(Wall(), (0, y))
            self.add_thing(Wall(), (self.width - 1, y))

        # Updates iteration start and end (with walls).
        self.x_start, self.y_start = (1, 1)
        self.x_end, self.y_end = (self.width - 1, self.height - 1)

    def add_observer(self, observer):
        """Adds an observer to the list of observers.
        An observer is typically an EnvGUI.

        Each observer is notified of changes in move_to and add_thing,
        by calling the observer's methods thing_moved(thing)
        and thing_added(thing, loc)."""
        self.observers.append(observer)

    def turn_heading(self, heading, inc):
        """Return the heading to the left (inc=+1) or right (inc=-1) of heading."""
        return turn_heading(heading, inc)


class Obstacle(Thing):
    """Something that can cause a bump, preventing an agent from
    moving into the same square it's in."""
    pass


class Wall(Obstacle):
    pass


# ______________________________________________________________________________


class GraphicEnvironment(XYEnvironment):
    def __init__(self, width=10, height=10, boundary=True, color={}, display=False):
        """Define all the usual XYEnvironment characteristics,
        but initialise a BlockGrid for GUI too."""
        super().__init__(width, height)
        self.grid = BlockGrid(width, height, fill=(200, 200, 200))
        if display:
            self.grid.show()
            self.visible = True
        else:
            self.visible = False
        self.bounded = boundary
        self.colors = color

    def get_world(self):
        """Returns all the items in the world in a format
        understandable by the ipythonblocks BlockGrid."""
        result = []
        x_start, y_start = (0, 0)
        x_end, y_end = self.width, self.height
        for x in range(x_start, x_end):
            row = []
            for y in range(y_start, y_end):
                row.append(self.list_things_at((x, y)))
            result.append(row)
        return result

    """
    def run(self, steps=1000, delay=1):
        "" "Run the Environment for given number of time steps,
        but update the GUI too." ""
        for step in range(steps):
            sleep(delay)
            if self.visible:
                self.reveal()
            if self.is_done():
                if self.visible:
                    self.reveal()
                return
            self.step()
        if self.visible:
            self.reveal()
    """

    def run(self, steps=1000, delay=1):
        """Run the Environment for given number of time steps,
        but update the GUI too."""
        for step in range(steps):
            self.update(delay)
            if self.is_done():
                break
            self.step()
        self.update(delay)

    def update(self, delay=1):
        sleep(delay)
        self.reveal()

    def reveal(self):
        """Display the BlockGrid for this world - the last thing to be added
        at a location defines the location color."""
        self.draw_world()
        # wait for the world to update and
        # apply changes to the same grid instead
        # of making a new one.
        clear_output(1)
        self.grid.show()
        self.visible = True

    def draw_world(self):
        self.grid[:] = (200, 200, 200)
        world = self.get_world()
        for x in range(0, len(world)):
            for y in range(0, len(world[x])):
                if len(world[x][y]):
                    self.grid[y, x] = self.colors[world[x][y][-1].__class__.__name__]

    def conceal(self):
        """Hide the BlockGrid for this world"""
        self.visible = False
        display(HTML(''))

---

# Aquí Actualización 1

---

In [ ]:
# ACTUALIZACION 1
from random import choice


class Trash(Thing):
    pass

class EnergeticRumbaCleaner(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def clean(self, thing):
        if isinstance(thing, Trash):
            return True
        return False
    
        
def program(percepts):
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Trash):
            return 'clean'
        if isinstance(p,Bump) or isinstance(p, Wall): # then check if you are at an edge and have to turn
            print('The EnergeticRumbaCleaner found a wall or something')
            turn = False
            choice = random.choice((1,2));
        else:
            choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'

class Room2D(GraphicEnvironment):
    def percept(self, agent):
        things = self.list_things_at(agent.location)
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1 #1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1 #1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
        if any(isinstance(x,Wall) for x in self.list_things_at(loc)):
            things.append(Wall())
            
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "clean":
            items = self.list_things_at(agent.location, tclass=Trash)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        no_cleanables = not any(isinstance(thing, Trash) for thing in self.things)
        broken_agents = not any(agent.is_alive() for agent in self.agents)
        return broken_agents or no_cleanables
     

In [ ]:
# Actualizacion 1
from ipythonblocks import BlockGrid
import random
import copy
import collections
import numbers
from time import sleep
from IPython.display import HTML, display, clear_output

#Aquí su código
room=Room2D(11,11, color={'EnergeticRumbaCleaner': (0,0,255), 'Wall': (0,0,0),  'Trash': (255, 0, 0)}) # park width is set to 5, and height to 20
cleaner = EnergeticRumbaCleaner(program)
trash = Trash()
room.add_thing(cleaner, [1,1])
room.add_thing(trash, [5,5])
room.add_thing(Wall(), [1,5])
room.add_thing(Wall(), [3,6])
room.add_thing(Wall(), [8,2])
room.add_thing(Wall(), [3,3])
room.add_thing(Wall(), [8,8])
room.add_walls()
print("cleaner started at [0,0], facing down. Let's see if it found any trash!")
room.run(20)

---

# Aquí Actualización 2

---

In [ ]:
# ACTUALIZACION 2
from random import choice

class Trash(Thing):
    pass

class Wall(Thing):
    pass

class Way(Thing):
    pass

class Vertex():
    def __init__(self, x, y, things = list()):
        """Remember the x and y are flipped in this grid (why god why) in the array
        x goes first then y
        """
        self.x = x 
        self.y = y
        self.parent_x = None
        self.parent_y = None
        # We avoid visiting walls by marking them as visted from the start =)
        self.visited = True if any(isinstance(t, Wall) for t in things) else False
        # This is an array with all the things in this cell
        self.things = things
        
class EnergeticRumbaCleaner(Agent):
    location = [1,1]
    direction = Direction("right")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def clean(self, thing):
        if isinstance(thing, Trash) or isinstance(thing, Way):
            return True
        return False
        
def program(percepts):
    choice = 0
    for p in percepts:
        if isinstance(p, Trash):
            return 'clean'
        if isinstance(p, Way):
            return 'moveforward'
    if choice == 0:
        return 'turnright'
    
class Room2D(GraphicEnvironment):
    def percept(self, agent):
        things = list()
        for x in range(len(self.list_things_at(agent.location))):
            if isinstance(self.list_things_at(agent.location)[x], Trash):
                things.append(self.list_things_at(agent.location)[x])
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1 #1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1 #1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
            
        if any(isinstance(x,Way) for x in self.list_things_at(loc)):
            things.append(Way())
            
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            items = self.list_things_at(agent.location, tclass=Way)
            if len(items) != 0:
                if agent.clean(items[0]):
                    self.delete_thing(items[0])
            
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "clean":
            print("ojo")
            items = self.list_things_at(agent.location, tclass=Trash)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    self.is_done()
                    
    def is_done(self):
        no_cleanables = not any(isinstance(thing, Trash) for thing in self.things)
        broken_agents = not any(agent.is_alive() for agent in self.agents)
        return broken_agents or no_cleanables
     
    
def breadth_first_search(room, sourceX, sourceY):
    "Search shallowest nodes in the search tree first."
    
    node = Vertex(sourceX, sourceY, room.list_things_at([sourceX, sourceY]))
    if any(isinstance(x, Trash) for x in room.list_things_at([sourceX, sourceY])):
        return node
    frontier = [node]
    while frontier:
        node = frontier.pop()
        Neighbors = GetNeighbors(node)
        for neighbor in Neighbors:
            if any(isinstance(x, Trash) for x in room.list_things_at([neighbor.x,neighbor.y])):
                return neighbor
            if not neighbor.visited:
                neighbors = True
                frontier.insert(0,neighbor)
    return failure

def GetNeighbors(vertice, width = 10, height = 10):
    neighbors = []
    if(vertice.x-1 >= 0):
        leftX = Vertex(vertice.x-1, vertice.y)
        leftX.parent_x = vertice
        neighbors.append(leftX)
    if(vertice.x+1 < width):
        rightX = Vertex(vertice.x+1, vertice.y)
        rightX.parent_x = vertice
        neighbors.append(rightX)
    if(vertice.y-1 >= 0):
        aboveY = Vertex(vertice.x, vertice.y-1)
        aboveY.parent_y = vertice
        neighbors.append(aboveY)
    if(vertice.y+1 < height):
        belowX = Vertex(vertice.x, vertice.y+1)
        belowX.parent_y = vertice
        neighbors.append(belowX)
    return neighbors
                  
def get_path(vertice, path, sourceX, sourceY):
    if vertice is not None:
        path.append(vertice)
    if vertice.x == sourceX and vertice.y == sourceY:
        return path
    if vertice.parent_x is not None:
        return get_path(vertice.parent_x, path, sourceX, sourceY)
    else:
        return get_path(vertice.parent_y, path, sourceX, sourceY)

In [ ]:
# Actualizacion 2
#Aquí su código
from ipythonblocks import BlockGrid
import random
from numpy import random
import copy
import collections
import numbers
from time import sleep
from IPython.display import HTML, display, clear_output

print("ahoy")

room=Room2D(11,11, color={'EnergeticRumbaCleaner': (0,0,255), 'Wall': (0,0,0),  'Trash': (255, 0, 0), 'Way':(130, 1, 130)}) # park width is set to 5, and height to 20
cleaner = EnergeticRumbaCleaner(program)

trashX = random.randint(6)+2
trashY = random.randint(6)+2

room.add_thing(Trash(), [trashX, trashY])
trash = breadth_first_search(room, 1, 1)
way = get_path(trash,[], 1, 1)
for x in range (len(way)):
    room.add_thing(Way(), [way[x].x, way[x].y])
room.add_thing(cleaner, [1,1])
print("cleaner started at [0,0], facing down. Let's see if it found any trash!")
room.add_thing(Trash(), [trashX,trashY])
room.run()

In [26]:
from random import choice

class Trash(Thing):
    pass

class Wall(Thing):
    pass

class Way(Thing):
    pass

class Vertex():
    def __init__(self, x, y, things = list()):
        """Remember the x and y are flipped in this grid (why god why) in the array
        x goes first then y
        """
        self.x = x 
        self.y = y
        self.parent_x = None
        self.parent_y = None
        # We avoid visiting walls by marking them as visted from the start =)
        self.visited = True if any(isinstance(t, Wall) for t in things) else False
        # This is an array with all the things in this cell
        self.things = things
        
class EnergeticRumbaCleaner(Agent):
    location = [1,1]
    direction = Direction("right")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (i.e. valid destination location)'''
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def clean(self, thing):
        if isinstance(thing, Trash) or isinstance(thing, Way):
            return True
        return False
        
def program(percepts):
    choice = 0
    for p in percepts:
        if isinstance(p, Trash):
            return 'clean'
        if isinstance(p, Way):
            return 'moveforward'
    if choice == 0:
        return 'turnright'
    
class Room2D(GraphicEnvironment):
    def percept(self, agent):
        things = list()
        for x in range(len(self.list_things_at(agent.location))):
            if isinstance(self.list_things_at(agent.location)[x], Trash):
                things.append(self.list_things_at(agent.location)[x])
        loc = copy.deepcopy(agent.location) # find out the target location
        #Check if agent is about to bump into a wall
        if agent.direction.direction == Direction.R:
            loc[0] += 1 #1
        elif agent.direction.direction == Direction.L:
            loc[0] -= 1
        elif agent.direction.direction == Direction.D:
            loc[1] += 1 #1
        elif agent.direction.direction == Direction.U:
            loc[1] -= 1
            
        if any(isinstance(x,Way) for x in self.list_things_at(loc)):
            things.append(Way())
            
        if not self.is_inbounds(loc):
            things.append(Bump())
        return things
    
    def execute_action(self, agent, action):
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            items = self.list_things_at(agent.location, tclass=Way)
            if len(items) != 0:
                if agent.clean(items[0]):
                    self.delete_thing(items[0])
            
            print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
            agent.moveforward()
        elif action == "clean":
            print("ojo")
            items = self.list_things_at(agent.location, tclass=Trash)
            if len(items) != 0:
                if agent.clean(items[0]):
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    if not self.is_done():
                        trash = breadth_first_search(room, agent.location[0], agent.location[1])
                        way = get_path(trash,[], agent.location[0], agent.location[1])
                        for i in range (len(way)):
                            room.add_thing(Way(), [way[i].x, way[i].y])

                        
                    
    def is_done(self):
        no_cleanables = not any(isinstance(thing, Trash) for thing in self.things)
        broken_agents = not any(agent.is_alive() for agent in self.agents)
        return broken_agents or no_cleanables
     
    
def breadth_first_search(room, sourceX, sourceY):
    "Search shallowest nodes in the search tree first."
    
    node = Vertex(sourceX, sourceY, room.list_things_at([sourceX, sourceY]))
    if any(isinstance(x, Trash) for x in room.list_things_at([sourceX, sourceY])):
        return node
    frontier = [node]
    while frontier:
        node = frontier.pop()
        Neighbors = GetNeighbors(node)
        for neighbor in Neighbors:
            if any(isinstance(x, Trash) for x in room.list_things_at([neighbor.x,neighbor.y])):
                return neighbor
            if any(isinstance(x, Wall) for x in room.list_things_at([neighbor.x,neighbor.y])) and not neighbor.visited:
                #print("Debug")
                neighbor.visited = True
                continue
            if not neighbor.visited:
                neighbors = True
                frontier.insert(0,neighbor)
    return failure

def GetNeighbors(vertice, width = 10, height = 10):
    neighbors = []
    if(vertice.x-1 >= 0):
        leftX = Vertex(vertice.x-1, vertice.y)
        leftX.parent_x = vertice
        neighbors.append(leftX)
    if(vertice.x+1 < width):
        rightX = Vertex(vertice.x+1, vertice.y)
        rightX.parent_x = vertice
        neighbors.append(rightX)
    if(vertice.y-1 >= 0):
        aboveY = Vertex(vertice.x, vertice.y-1)
        aboveY.parent_y = vertice
        neighbors.append(aboveY)
    if(vertice.y+1 < height):
        belowX = Vertex(vertice.x, vertice.y+1)
        belowX.parent_y = vertice
        neighbors.append(belowX)
    return neighbors
                  
def get_path(vertice, path, sourceX, sourceY):
    if vertice is not None:
        path.append(vertice)
    if vertice.x == sourceX and vertice.y == sourceY:
        return path
    if vertice.parent_x is not None:
        return get_path(vertice.parent_x, path, sourceX, sourceY)
    else:
        return get_path(vertice.parent_y, path, sourceX, sourceY)

In [27]:
# Actualizacion 3
#Aquí su código
from ipythonblocks import BlockGrid
import random
from numpy import random
import copy
import collections
import numbers
from time import sleep
from IPython.display import HTML, display, clear_output

print("ahoy")

room=Room2D(11,11, color={'EnergeticRumbaCleaner': (0,0,255), 'Wall': (0,0,0),  'Trash': (255, 0, 0), 'Way':(130, 1, 130)}) # park width is set to 5, and height to 20
cleaner = EnergeticRumbaCleaner(program)

# Si quieres más paredes cambia la definicion de walls y descomenta el add_walls.
# considere que esto aumentara la complejidad exponencialmente.

walls = [ [6,2], [5,2], [3,2], [2,2], 
          [1,2], [1,5], [8,2], [3,3]]
#room.add_walls()

#si quieres más basuras cambia el valor de t:
t = 3
for j in range(t): 

    trashX = random.randint(6)+2
    trashY = random.randint(6)+2

    while [trashX, trashY] in walls:
        trashX = random.randint(6)+2
        trashY = random.randint(6)+2

    room.add_thing(Trash(), [trashX, trashY])

for wall in walls:
    room.add_thing(Wall(), wall)
trash = breadth_first_search(room, 1, 1)
way = get_path(trash,[], 1, 1)
for i in range (len(way)):
    room.add_thing(Way(), [way[i].x, way[i].y])
room.add_thing(cleaner, [1,1])
print("cleaner started at [0,0], facing down. Let's see if it found any trash!")
room.add_thing(Trash(), [trashX,trashY])
room.run()

ahoy
cleaner started at [0,0], facing down. Let's see if it found any trash!


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,


EnergeticRumbaCleaner decided to turnright at location: [1, 1]


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,


EnergeticRumbaCleaner decided to turnright at location: [1, 1]


...

ojo
EnergeticRumbaCleaner cleaned Trash at location: [7, 3]


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,


ojo
EnergeticRumbaCleaner cleaned Trash at location: [7, 3]


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,


,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
,,,,,,,,,,
